# Описание проекта

- Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
- Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
- Постройте модель со значением метрики качества F1 не меньше 0.75.

# Описание данных
Столбец text в нём содержит текст комментария, а toxic — целевой признак.

# 1.0 Подготовка:

In [1]:
!pip install pymystem3

In [2]:
import nltk
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import re
from sklearn.pipeline import Pipeline
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mmaximmaximovgmail.com/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mmaximmaximovgmail.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mmaximmaximovgmail.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Откроем таблицу двумя путями
server_data = '/datasets/toxic_comments.csv'
local_data = '/Users/mmaximmaximovgmail.com/Desktop/DS/project 13/toxic_comments.csv'

try:
    toxic = pd.read_csv(server_data)
except:
    toxic = pd.read_csv(local_data)

In [4]:
# Раскроем таблицу и кол-во дубликатов
display(toxic.head())
print()
toxic.info()
print()
display(toxic.duplicated().sum())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB



0

### Мини-вывод 1.0
- Выгрузил данные.
- Проверил нет ли пропусков в колонках
- Проверил дубликаты.
- С данными все хорошо, можем обучать.

# 2.0 Обучение:

In [5]:
%%time
# Лемматизирую, очищаем текст для подгатовки обучения
# Так же делаем текст в нижний индекс
# Выбираю a-zA-Z, так как текст у нас на англ языке

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text_spl = text.split()
    lemma_word = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text_spl] 
    text_fin = ' '.join(lemma_word)
    return text_fin

toxic['text'] = toxic['text'].apply(lemmatize)
toxic.text

CPU times: user 11min 8s, sys: 34.7 s, total: 11min 42s
Wall time: 11min 43s


0         explanation why the edits make under my userna...
1         d aww he match this background colour i m seem...
2         hey man i m really not try to edit war it s ju...
3         more i can t make any real suggestion on impro...
4         you sir be my hero any chance you remember wha...
                                ...                        
159287    and for the second time of ask when your view ...
159288    you should be ashamed of yourself that be a ho...
159289    spitzer umm there no actual article for prosti...
159290    and it look like it be actually you who put on...
159291    and i really don t think you understand i come...
Name: text, Length: 159292, dtype: object

In [6]:
# Делю данные на выборки и заранее присваиваю знаение cv
features = toxic.drop('toxic', axis=1)
target = toxic['toxic']

ft_train, ft_valid, tg_train, tg_valid = train_test_split(
    features, target, test_size = 0.1, random_state=12345
)

ft_valid, ft_test, tg_valid, tg_test = train_test_split(
    ft_valid, tg_valid, test_size = 0.25, random_state=12345
)

count_cv = 3

ft_train.shape, ft_valid.shape, ft_test.shape

((143362, 2), (11947, 2), (3983, 2))

In [7]:
# Создам стоп слово, для англ, так как у нас данные на английском языке (возможно на британском)
nltk.download('stopwords')
nltk.download('wordnet')
wordnet = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mmaximmaximovgmail.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mmaximmaximovgmail.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Векторизация TF&IDF
count_tf_idf = TfidfVectorizer(stop_words = wordnet)
ft_train = count_tf_idf.fit_transform(ft_train['text'])
ft_valid = count_tf_idf.transform(ft_valid['text'])
ft_test = count_tf_idf.transform(ft_test['text'])
display(ft_train.shape)
display(ft_valid.shape)
display(ft_test.shape)

(143362, 142209)

(11947, 142209)

(3983, 142209)

In [9]:
# Начну с LogisticRegression
model = LogisticRegression(random_state=12345, solver='liblinear')
result_f1 = cross_val_score(
    model, 
    ft_train,
    tg_train,
    cv = count_cv,
    scoring='f1'
).mean()
print('Результат F1:', result_f1)

Результат F1: 0.7126656786715619


### Мини вывод 2.0
- Выполнил первичное обучение модели LogisticRegression.
- Получил результат F1 со значением 0.7126656786715619.
- Данный результат не устраивает, так как нужно высше '0.75'.
- Поэтому вношу поправки в первичное обучение.

# 2.1 Улучшаем первичную модель:

In [10]:
# Поробую сначала автоматическую корректировку с помощью class_weight='balanced'

model = LogisticRegression(class_weight='balanced', random_state=12345, solver='liblinear')
result_f1_balanced = cross_val_score(
    model, 
    ft_train,
    tg_train,
    cv = count_cv,
    scoring='f1'
).mean()
print('Результат F1:', result_f1_balanced)

Результат F1: 0.7434711197771877


In [11]:
# Теперь попробую внести определенные параметры для модели через Pipeline
pipeline = Pipeline(
    [
        ('model', LogisticRegression(random_state=12345, solver='liblinear'))
    ]
)

params = {
    'model__penalty' : ('l2', 'elasticnet'),
    'model__C' : list(range(1,10))
}

grid_search = GridSearchCV(pipeline, param_grid = params, n_jobs=-1, verbose=1, scoring='f1', cv=count_cv)
grid_search.fit(ft_train, tg_train)
print("Лучшее значение: %0.3f" % grid_search.best_score_)
print("Лучшие параметры:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/mmaximmaximovgmail.com/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 450, in _check_solve

Лучшее значение: 0.760
Лучшие параметры:
	model__C: 8
	model__penalty: 'l2'


In [12]:
# Теперь обучим модель с точными данными
model = LogisticRegression(random_state=123456, C = 9, penalty = 'l2', solver='liblinear', max_iter=100)
model.fit(ft_train, tg_train)
pred = model.predict(ft_valid)
f1_score(tg_valid, pred)

0.786764705882353

# 3.0 Тестовая модель:

In [13]:
# И на последок обучаем тестовую модель
model = LogisticRegression(random_state=123456, C = 9, penalty = 'l2', solver='liblinear', max_iter=100)
model.fit(ft_train, tg_train)
pred_test = model.predict(ft_test)
f1_score(tg_test, pred_test)

0.7645348837209303

# 4.0 Вывод:

    В данной работе была проведена работа:
1. Сделал проверку всех данных и убедился, что с ними все хорошо и можно работать.
2. Далее убрал лишние символы, пропуски, знаки в тексте и перевел в нижний регистр.
3. Создал стоп слово через WordNetLemmatizer и векторизацию TF&IDF
4. Выполнил первую модель, которая показывает не достаточный результат
5. После подбора правильных параметров с помощью LogisticRegression, я получил результат: 0.7645348837209303